In [1]:
from datetime import datetime 

import pandas as pd
import numpy as np
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AR

/home/chiehcheng/.local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

## Load training data

In [3]:
bj_aq1 = pd.read_csv('data/bj/beijing_17_18_aq.csv')
bj_aq2 = pd.read_csv('data/bj/beijing_201802_201803_aq.csv')
bj_aq3 = pd.read_csv('data/bj/bj_aq_0401_0423.csv') # data from api
ld_aq1 = pd.read_csv('data/ld/London_historical_aqi_forecast_stations_20180331.csv')
ld_aq2 = pd.read_csv('data/ld/ld_aq_0331_0423.csv') # data from api

In [4]:
ld_aq2.columns = bj_aq3.columns = ['id', 'stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']
ld_aq1.columns = ['unused', 'utc_time', 'stationId', 'PM2.5', 'PM10', 'NO2']

In [5]:
bj_aq = pd.concat([bj_aq1, bj_aq2, bj_aq3])
bj_aq['utc_time'] = pd.to_datetime(bj_aq['utc_time'])
bj_aq = bj_aq[['utc_time', 'stationId', 'PM2.5', 'PM10', 'O3']]
bj_aq.index = bj_aq['utc_time']
bj_aq.head()

,utc_time,stationId,PM2.5,PM10,O3
utc_time,,,,,
2017-01-01 14:00:00,2017-01-01 14:00:00,aotizhongxin_aq,453.0,467.0,3.0
2017-01-01 15:00:00,2017-01-01 15:00:00,aotizhongxin_aq,417.0,443.0,2.0
2017-01-01 16:00:00,2017-01-01 16:00:00,aotizhongxin_aq,395.0,467.0,3.0
2017-01-01 17:00:00,2017-01-01 17:00:00,aotizhongxin_aq,420.0,484.0,3.0
2017-01-01 18:00:00,2017-01-01 18:00:00,aotizhongxin_aq,453.0,520.0,4.0


In [6]:
ld_aq = pd.concat([ld_aq1, ld_aq2])
ld_aq['utc_time'] = pd.to_datetime(ld_aq['utc_time'])
ld_aq = ld_aq[['utc_time', 'stationId', 'PM2.5', 'PM10']]
ld_aq.index = ld_aq['utc_time']
ld_aq.head()

,utc_time,stationId,PM2.5,PM10
utc_time,,,,
2017-01-01 00:00:00,2017-01-01 00:00:00,CD1,40.0,44.4
2017-01-01 01:00:00,2017-01-01 01:00:00,CD1,31.6,34.4
2017-01-01 02:00:00,2017-01-01 02:00:00,CD1,24.7,28.1
2017-01-01 03:00:00,2017-01-01 03:00:00,CD1,21.2,24.5
2017-01-01 04:00:00,2017-01-01 04:00:00,CD1,24.9,23.0


## handle missing value

In [7]:
%%time

idx = pd.date_range(bj_aq.index.min(), bj_aq.index.max(), freq='H')
bj_aq_dict = dict()
grouped = bj_aq.groupby('stationId')
for name, group in grouped:
    df = group.drop_duplicates('utc_time', keep='last')
    df = df.reindex(idx)
    df.interpolate(method='slinear', inplace=True)
    bj_aq_dict[name] = df[['PM2.5', 'PM10', 'O3']]

CPU times: user 1.51 s, sys: 31.8 ms, total: 1.54 s
Wall time: 1.56 s


In [8]:
%%time

idx = pd.date_range(ld_aq.index.min(), ld_aq.index.max(), freq='H')
ld_aq_dict = dict()
grouped = ld_aq.groupby('stationId')
for name, group in grouped:
    df = group.drop_duplicates('utc_time', keep='last')
    df = df.reindex(idx)
    df.interpolate(method='slinear', inplace=True)
    ld_aq_dict[name] = df[['PM2.5', 'PM10']]

CPU times: user 856 ms, sys: 7.37 ms, total: 863 ms
Wall time: 862 ms


## Train the AR model and use it to predict

In [9]:
%%time

ar_models = dict()
results = dict()

#start = datetime(2018,4,23,15)
end = datetime(2018,4,25,23)

for _dict in [bj_aq_dict, ld_aq_dict]:
    for station, aq_df in _dict.items():
        ar_models[station] = dict()
        results[station] = dict()

        for pollutant in aq_df:

            # prevent data of some station is empty like 'zhiwuyuan_aq'
            if len(aq_df[pollutant].dropna()) is 0: 
                continue
            
            model = AR(aq_df[pollutant], missing='drop')
            ar_models[station][pollutant] = model_fit = model.fit()

            #print(station, pollutant)

            # start forecasting from the next hour of traing data to the end datetime
            predictions = model_fit.predict(start=len(aq_df[pollutant].dropna()), end=end, dynamic=False)
            results[station][pollutant] = predictions

CPU times: user 26.7 s, sys: 20.1 s, total: 46.8 s
Wall time: 12.5 s


## Output the result to submission file

In [10]:
submission = pd.read_csv('data/sample_submission.csv')
submission['PM2.5'] = submission['PM2.5'].astype('float64')
submission['PM10'] = submission['PM10'].astype('float64')
submission['O3'] = submission['O3'].astype('float64')

test_date = pd.date_range(datetime(2018,4,24,0), datetime(2018,4,25,23), freq='H')

submission.index = submission['test_id']
for index, row in submission.iterrows():
    station, num = index.split('#')
    date = test_date[int(num)]
    submission.at[index, 'O3'] = results[station]['O3'][date] if 'O3' in results[station] else 0
    submission.at[index, 'PM2.5'] = results[station]['PM2.5'][date]
    submission.at[index, 'PM10'] = results[station]['PM10'][date]

In [11]:
submission = submission.reset_index(drop=True)
submission.to_csv('ar_submission.csv', index=False)